Contra las mil cabezas de la Hidra
============================


¿Cómo llevar registro de experimentos con ayuda de MLFlow y Hydra?
-----------------------------------------------------------------------------------------------------

## ¿Quién soy?

<div style="text-align:center;">
    <h4 style="font-size:1.5em;margin:5px;">Cristian Cardellino</h4>
    <h5 style="font-style:normal;font-size:1em;margin:5px;">Research en Mercado Libre - Docente en UNC</h5>
    <div style="display:inline-block;margin-right:20px;">
        <img src="./img/me.jpg" style="height:10em;width:auto;"/>
    </div>
    <h6 style="font-style:normal;font-size:0.9em;margin:5px;">
        <a href="https://twitter.com/crscardellino" style="color:royalblue;" target="_blank">@crscardellino</a> -
        <a href="https://crscardellino.ar" style="color:royalblue;" target="_blank">https://crscardellino.ar</a>
    </h6>
</div>

## Esquema de la charla

1. [MLFlow](#MLFlow)
1. [Hydra](#Hydra)
1. [MLFlow + Hydra: Un framework de experimentación](#MLFlow-+-Hydra:-Un-framework-de-experimentación)

# MLFlow

## ¿Qué es MLFlow?

Es una [plataforma de código abierto para trabajar con el **ciclo de vida en aplicaciones de aprendizaje automático**](https://mlflow.org). Entre sus funcionalidad se destacan:

* Lleva registro de experimentos (local o remoto), para comparar hiperparémetros y resultados.
* Empaqueta código de manera que sea posible de compartir y reutilizar.
* Administra y despliega modelos de distintos frameworks de Machine Learning para servirlos online.
* Provee un modelo central para colaborar durante el desarrollo de una aplicación de aprendizaje automático.

En esta charla nos centraremos en el primer punto, i.e. el registro de experimentos, sin embargo les invito a leer la [documentación](https://mlflow.org/docs/latest/index.html) sobre los demás temas, si les interesan, la cuál es muy buena.

## Instalación de MLFlow

Para instalar con `pip`:

    $ pip install mlflow

Para instalar con `conda`:

    $ conda install mlflow -c conda-forge

In [ ]:
import mlflow
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

## Corriendo un experimento sencillo

Vamos a trabajar con el problema de clasificación de calidad de vinos del [conjunto de datos del repositorio UCI](https://archive.ics.uci.edu/ml/datasets/Wine), que se encuentra disponible en el archivo `./data/wines-data.csv`. Este ya está dividido en `train/test/validation`.

In [ ]:
data = pd.read_csv('./data/wines-data.csv')
display(data.head())

train_data = data.loc[data['Split'] == 'train'].iloc[:, 2:].values
train_target = data.loc[data['Split'] == 'train', 'Quality'].values

val_data = data.loc[data['Split'] == 'validation'].iloc[:, 2:].values
val_target = data.loc[data['Split'] == 'validation', 'Quality'].values

In [ ]:
def run_experiment(solver, penalty, C):
    with mlflow.start_run():
        mlflow.log_params({
            'solver': solver,
            'penalty': penalty,
            'C': C
        })

        clf = LogisticRegression(
            penalty=penalty,
            solver=solver,
            C=C
        ).fit(train_data, train_target)

        val_preds = clf.predict(val_data)
        accuracy = accuracy_score(val_target, val_preds)
        f1_per_class = f1_score(val_target, val_preds, average=None)
        f1_macro = pd.Series(f1_per_class).mean()

        mlflow.log_metrics({
            'accuracy': accuracy,
            'f1_macro': f1_macro
        })

        for class_idx, class_f1 in enumerate(f1_per_class, start=1):
            mlflow.log_metric(f"f1_quality_{class_idx}", class_f1)

## Experiment vs. Run

MLFlow tiene dos entidades principales a la hora de trabajar. Los [`experiments`](https://mlflow.org/docs/latest/tracking.html#organizing-runs-in-experiments) y los [`runs`](https://mlflow.org/docs/latest/tracking.html#logging-data-to-runs). En sí, al correr un experimento, lo que hacemos es hacer un *run* donde hace el registro de los datos (parámetros y métricas), y varios de estos *run* (con diferentes datos) conforman un experimento.

In [ ]:
solver = 'liblinear'  # Type of solver for the LR algorithm
penalty = 'l2'  # Type of regularization penalty (depends on the solver)
C = 1/1e-3  # This is the inverse of the regularization parameter

run_experiment(solver, penalty, C)

## Revisando la UI de MLFlow

Una vez que corremos el experimento, el siguiente punto es la verificación de los resultados en la UI nativa de MLFlow. Para ello basta con iniciar dicha UI en una terminal mediante el siguiente comando:

    (venv) $ mlflow ui # [--host 127.0.0.1] [--port 5000]

## Realizando una búsqueda exhaustiva

Correr un experimento y hacer un registro de parámetros y métricas es sólo lo básico que ofrece el `Tracking` de MLFlow. La verdadera potencia reside en poder realizar varios runs para comparar.

In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'solver': ['liblinear', 'saga'],
    'penalty': ['l1', 'l2'],
    'C': [1/1e-2, 1/1e-3, 1/1e-4]
}

for parameters in ParameterGrid(param_grid):
    run_experiment(**parameters)

## Personalizando experiments y runs

Por defecto, MLFlow guarda todos los runs que se corren en un experimento por defecto que suele llevar el nombre `Default` y el ID `0`. Cuando queremos correr distintos experimentos (e.g. para distintos conjuntos de datos o para distintos tipos de clasificador), podemos hacer múltiples experimentos configurando correctamente los parámetros.

In [ ]:
def run_experiment(solver, penalty, C, run_name):
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params({
            'solver': solver,
            'penalty': penalty,
            'C': C
        })

        clf = LogisticRegression(
            penalty=penalty,
            solver=solver,
            C=C
        ).fit(train_data, train_target)

        val_preds = clf.predict(val_data)
        accuracy = accuracy_score(val_target, val_preds)
        f1_per_class = f1_score(val_target, val_preds, average=None)
        f1_macro = pd.Series(f1_per_class).mean()

        mlflow.log_metrics({
            'accuracy': accuracy,
            'f1_macro': f1_macro
        })

        for class_idx, class_f1 in enumerate(f1_per_class, start=1):
            mlflow.log_metric(f"f1_quality_{class_idx}", class_f1)

In [ ]:
mlflow.set_experiment(experiment_name='personalized_experiment')

for params in ParameterGrid(param_grid):
    run_name = f"solver:{params['solver']}_penalty:{params['penalty']}_reg:{params['C']}"
    run_experiment(**params, run_name=run_name)

## Editando la descripción

- Una de las cosas que permite la UI de MLFlow es editar una descripción sobre determinada corrida de un experimento. 
- Esto puede ser útil para guardar cosas más complejas que una métrica, pero que no requieran tanto espacio como un artefacto y sean fáciles de acceder. 
- Para ello podemos reescribir un `tag` especial de MLFlow que indica que las notas de determinada corrida se guardarán con ciertos valores. 
  - Por ejemplo, podemos utilizarlos para guardar el **reporte de clasificación** de Scikit-Learn (¡y funciona con Markdown!).

In [ ]:
from sklearn.metrics import classification_report

def run_experiment(solver, penalty, C):
    with mlflow.start_run():
        mlflow.log_params({
            'solver': solver,
            'penalty': penalty,
            'C': C
        })

        clf = LogisticRegression(
            penalty=penalty,
            solver=solver,
            C=C
        ).fit(train_data, train_target)

        val_preds = clf.predict(val_data)
        accuracy = accuracy_score(val_target, val_preds)
        f1_per_class = f1_score(val_target, val_preds, average=None, zero_division=0)
        f1_macro = pd.Series(f1_per_class).mean()

        mlflow.log_metrics({
            'accuracy': accuracy,
            'f1_macro': f1_macro
        })

        for class_idx, class_f1 in enumerate(f1_per_class, start=1):
            mlflow.log_metric(f"f1_quality_{class_idx}", class_f1)

        run_report = classification_report(val_target, val_preds, zero_division=0)
        mlflow.set_tag("mlflow.note.content", 
                       "Reporte de clasificación para esta corrida:\n"
                       f"```\n{run_report}\n```")

In [ ]:
mlflow.set_experiment(experiment_name='description_experiment')

for parameters in ParameterGrid(param_grid):
    run_experiment(**parameters)

## Artefactos en MLFlow

- Una de las características más interesantes que brinda MLFlow es la capacidad de guardar **artefactos**. 
    - Un artefacto es un archivo (o directorio) que existen localmente y puede ser copiado a una unidad de almacenamiento definida por MLFlow. 
    - Dicha unidad puede ser local o remote (e.g. un Amazon S3). 
- En particular son útiles para guardar información extra que a veces puede ser necesaria.
    - Un archivo de predicciones con información extra como la probabilidad de cada clase.
    - Un archivo de predicciones erróneas que puede ser utilizado luego en análisis de error detallado.

In [ ]:
from pathlib import Path
from tempfile import TemporaryDirectory

def run_experiment(solver, penalty, C):
    with mlflow.start_run():
        mlflow.log_params({
            'solver': solver,
            'penalty': penalty,
            'C': C
        })

        clf = LogisticRegression(
            penalty=penalty,
            solver=solver,
            C=C
        ).fit(train_data, train_target)

        # Obtengo la probabilidad por clase de cada una de las instancias
        val_preds_probs = clf.predict_proba(val_data)
        val_preds = clf.predict(val_data)

        accuracy = accuracy_score(val_target, val_preds)
        f1_per_class = f1_score(val_target, val_preds, average=None)
        f1_macro = pd.Series(f1_per_class).mean()

        mlflow.log_metrics({
            'accuracy': accuracy,
            'f1_macro': f1_macro
        })

        for class_idx, class_f1 in enumerate(f1_per_class, start=1):
            mlflow.log_metric(f"f1_quality_{class_idx}", class_f1)

        predictions_features = pd.DataFrame(val_data, columns=data.columns[2:])
        target_predictions = pd.Series(val_target, name="Quality")
        predictions_probs = pd.DataFrame(val_preds_probs,
                                         columns=[f"Quality {i+1} Prediction Probability" for i in range(3)])
        predictions_dataset = pd.concat([target_predictions, predictions_probs, predictions_features],
                                        axis=1)
        with TemporaryDirectory() as tmpdir:
            predictions_path = Path(tmpdir) / 'predictions.csv'
            predictions_dataset.to_csv(predictions_path, index=False)
            mlflow.log_artifact(predictions_path)

In [ ]:
mlflow.set_experiment(experiment_name='artifact_experiment')

for parameters in ParameterGrid(param_grid):
    run_experiment(**parameters)

# Hydra

## ¿Qué es hydra?

Es un [framework open source para **simplificar el desarrollo de aplicaciones de investigación y aplicaciones de configuración compleja**](https://hydra.cc/). Algunas características:

* Configuración jerárquica compuesta de múltiples fuentes.
* La configuración puede ser especificada y sobre-escrita desde la línea de comando.
* Correr la aplicación de manera local o remota.
* Correr múltiples veces con diferentes argumentos desde un sólo comando.

Si bien en esta charla nos concentraremos en los dos primeros puntos, pueden leer la [documentación](https://hydra.cc/docs/intro/) para explorar todas las posibilidades que ofrece.

## Instalación de Hydra

Para instalar con `pip`:

    $ pip install hydra-core --upgrade

## Una aplicación sencilla

Empezamos mostrando una aplicación sencilla con un archivo de configuración. Hydra no se puede correr (al menos no trivialmente) en un notebook, así que armamos un pequeño paquete para correrlo. Este poseerá el programa principal y un directorio con la configuración:

In [1]:
!tree ./hydra_basic/

./hydra_basic/
├── conf
│   └── config.yaml
└── experiment.py

1 directory, 2 files


### Configuración

El archivo de configuración es un YAML.

In [2]:
!yq -C . < ./hydra_basic/conf/config.yaml

input:
  data_file: ./data/wines-data.csv
train:
  split: train
  model:
    penalty: l2
    solver: liblinear
    C: 1000
evaluation:
  split: validation


### Programa principal

Una aplicación Hydra se define mediante el decorador `@hydra.main` que es el que lee el archivo de configuración y lo transforma en un [`DictConfig`](https://omegaconf.readthedocs.io/en/2.2_branch/) (i.e. un diccionario de configuración).

In [ ]:
import hydra
from omegaconf import DictConfig, OmegaConf

@hydra.main(config_path='conf', config_name='config', version_base=None)
def main(cfg: DictConfig):
    print(OmegaConf.to_yaml(cfg))

### Ejecutando el programa

La celda anterior sólo es de muestra, en general no tiene sentido correr hydra desde un notebook. En la siguiente celda corremos la [aplicación de muestra](/edit/hydra_basic/experiment.py).

In [ ]:
!python ./hydra_basic/experiment.py

### Logs del programa

La aplicación en realidad se corre bajo un nuevo directorio que se crea al momento de ejecutar el programa y está por defecto en la dirección `./outputs/DATA/HOUR/`. En este caso podemos ver que una vez terminado de correr el programa tenemos un archivo con los logs del mismo

In [ ]:
!tree ./outputs

In [ ]:
!find ./outputs -type f -name "*.log" -exec cat {} \;

## Personalizando configuración via CLI

Parte de la potencia de Hydra reside en poder cambiar la configuración sin necesidad de modificar el archivo, y sólo mediante una interfaz de línea de comandos (CLI). Existen 3 opciones a la hora de sobreescribir:

- Sobreescribir un parámetro existente
- Crear un nuevo parámetro
- Realizar un *upsert* (i.e. sobreescribir si existe y crearlo sino).

A la hora de correr se hace escribiendo la configuración como una *lista de puntos*. Si se quiere agregar un nuevo valor, se pone el prefijo `+`, si se quiere hacer un *upsert* es `++`:

In [ ]:
%%bash

python ./hydra_basic/experiment.py \
    evaluation.split=test \
    +train.model.max_iter=10000 \
    ++train.model.solver=saga \
    ++train.model.random_state=42

## Archivo de configuración avanzado

Más allá de la base de configuración, hydra también soporta otras cosas en sus archivos de configuración que permiten mayor flexibilidad. Entre estas destacan:

* **Configuraciones requeridas:** Aquellas cuyos valores están faltantes y se requieren para continuar la computación. Estas se determinan con el valor especial `???`.
* **Interpolación de valores:** Cuando un valor de alguna configuración requiere de otro valor se pueden utilizar interpolaciones y acceder a la configuración requerida. Esto se hace mediante `${path.a.la.config}`.
* **Resolvers:** Estos definen funciones que se ejecutarán en el script python en tiempo de ejecución. Es algo muy poderoso, pero que también se debe utilizar con cuidado.

In [3]:
!yq -C . < ./hydra_advanced/conf/config.yaml

input:
  data_file: ???
  random_seed: 42
train:
  split: train
  model:
    penalty: l2
    solver: liblinear
    C: ${eval:1 / 1e-3}
    random_state: ${input.random_seed}
evaluation:
  split: validation


### Ejecutando el programa

Si revisamos la nueva [aplicación de muestra](/edit/hydra_advanced/experiment.py), veremos que tenemos una línea en particular que registra un nuevo *resolver* que lo que hace es evaluar la operación. En este caso, la única configuración que la utiliza es `train.split.C`. Por otro lado, requerimos del path al archivo de datos en este caso, y además vemos que la configuración `input.random_seed` es copiada en `train.model.random_state`.

In [ ]:
!python ./hydra_advanced/experiment.py input.data_file=./data/wines-data.csv

## Configuraciones complejas

Hydra permite ir más lejos y hacer configuraciones más complejas a través de múltiples archivos.

In [4]:
!tree ./hydra_complex/

./hydra_complex/
├── conf
│   ├── config.yaml
│   └── train
│       └── model
│           ├── logreg.yaml
│           └── svm.yaml
└── experiment.py

3 directories, 4 files


### Modelos con configuraciones propias

En nuestro ejemplo modificamos el parámetro `train.model` del archivo [`config.yaml`](/edit/hydra_complex/conf/config.yaml) y lo sustituimos por configuraciones propias en `conf/train/model`. Tenemos dos modelos y parámetros para los mismos: [`logreg.yaml`](/edit/hydra_complex/conf/train/model/logreg.yaml) y [`svm.yaml`](/edit/hydra_complex/conf/train/model/svm.yaml).

In [5]:
!yq -C . < ./hydra_complex/conf/config.yaml

defaults:
  - _self_
  - train/model: logreg
input:
  data_file: ???
  random_seed: 42
train:
  split: train
evaluation:
  split: validation


In [6]:
!yq -C . < ./hydra_complex/conf/train/model/logreg.yaml

module: ${eval:LogisticRegression}
params:
  penalty: l2
  solver: liblinear
  C: ${eval:1 / 1e-3}
  random_state: ${input.random_seed}


In [7]:
!yq -C . < ./hydra_complex/conf/train/model/svm.yaml

module: ${eval:LinearSVC}
params:
  penalty: l2
  loss: hinge
  C: ${eval:1 / 1e-3}
  random_state: ${input.random_seed}


### Ejecutando la aplicación

* Básicamente, lo que cambia es que en esta [nueva versión de la aplicación](/edit/hydra_complex/experiment.py), podemos elegir el tipo de clasificador (de las configuraciones disponibles, claro está) que queremos utilizar. 
* Por defecto, se utilizarán los datos de la configuración `logreg.yaml` puesto que se define en el archivo de configuración principal bajo la configuración especial `defaults`.
* El parámetro `_self_` indica la prioridad en caso de que haya configuraciones compartidas entre el archivo de configuración principal y alguno de los archivos por defecto, si `_self_` está al final entonces cualquier configuración compartida toma el valor del archivo de configuración.

In [ ]:
!python ./hydra_complex/experiment.py input.data_file=./data/wines-data.csv

Si queremos entrenar con el modelo `svm`, lo aclaramos al correr la configuración.

In [ ]:
%%bash

python ./hydra_complex/experiment.py \
    input.data_file=./data/wines-data.csv \
    train/model=svm \
    train.model.params.C='${eval:1/1e-4}'

# MLFlow + Hydra: Un framework de experimentación para Python

## ¿Cómo es mi framework de experimentación?

En el día a día utilizo MLFlow y Hydra para realizar una gran cantidad de experimentos. Para poder llevar registro de lo que busco con dichos experimentos, con la ayuda de Hydra y MLFlow diseñé un patrón que me sirve para organizarme:

* Cada experimento de MLFlow define una hipótesis de experimentación.
* Utilizo los nombres y, sobre todo, las descripciones de los experimentos para establecer la hipótesis que estoy investigando y no olvidarla. De esta forma entiendo cuál era el objetivo de un conjunto de experimentos realizados.
    * E.g. "Hipótesis: Utilizar más capas en el perceptrón multicapa produce overfitting."
* Los *runs* definen las configuraciones para rechazar o no dicha hipótesis.
* En cada *run* guardo información importante (en el nombre o descripción) de lo más interesante de dicho *run*.
    * E.g. dejo en claro el número de capas en el nombre y/o la descripción para poder diferenciarlo fácilmente.
* Durante el proceso de entrenamiento utilizo `mlflow.log_artifacts` y puedo guardar la configuración total y/o el modelo.
* Finalizados los runs, puedo ver si la hipótesis se rechaza.
    * Puedo utilizar la comparación (scatterplot) también para ver que features hacen diferencias.

## Ejemplo de una aplicación de experimentación

En el directorio `./mlflow_hydra` hay una aplicación completa de cómo utilizo yo MLFlow + Hydra (+ bonus de [Pytorch Lighning](https://pytorch-lightning.readthedocs.io/en/latest/)) en mi día a día.

In [8]:
!tree ./mlflow_hydra/

./mlflow_hydra/
├── conf
│   └── config.yaml
├── experiment.py
├── __init__.py
├── model.py
└── utils.py

1 directory, 5 files


### El archivo de configuración

En el [archivo de configuración](/edit/mlflow_hydra/conf/config.yaml) tenemos dos secciones principales:

1. La sección `input` define cuestiones más del experimento, en particular, además del path al archivo de datos que voy a estar utilizando, explicito cosas como la hipótesis (que es el nombre de un experimento en MLFlow), la descripción de dicha hipótesis y el nombre del `run` de MLFLow que utilizaré para destacar la información (parámetros) más importantes para un fácil acceso.
2. En la sección `train` tengo los detalles de implementación que usaré en el experimento, i.e. los hiperparámetros a modificar a lo largo de los diferentes `run`s de dicho experimento que ayudarán a rechazar o no la hipótesis.

In [9]:
!yq -C . < ./mlflow_hydra/conf/config.yaml

input:
  data_file: ???
  run_name: ???
  experiment_name: Hypothesis 1
  experiment_description: "**Hypothesis 1:** More layers increase the overfitting"
train:
  test_evaluation: false
  feature_scaling: true
  batch_size: 16
  epochs: 10
  early_stop: 3
  model:
    layers: [64]
    learning_rate: 1e-3
    l2_lambda: 1e-5
    activation: ${eval:torch.nn.ReLU}


### La aplicación del experimento

La [aplicación del experimento](/edit/mlflow_hydra/experiment.py) consta de una función `main` que se encargará de hacer el setup inicial de MLFLow, definiendo el nombre del experimento, la descripción, y guardando cosas como los hiperparámetros y el archivo de configuración. Además, la función `run_experiment` que simplemente corre el modelo de Pytorch Lightning (recomiendo leer la [documentación oficial](https://pytorch-lightning.readthedocs.io/en/latest/starter/introduction.html) para entender en más detalle el ejemplo), que tiene la habilidad de hacer un [binding con el mismo MLFlow](https://pytorch-lightning.readthedocs.io/en/latest/visualize/experiment_managers.html#mlflow) para llevar registro de como el modelo va aprendiendo a lo largo de las distintas épocas.

### Corriendo iteraciones del experimento para verificar la hipótesis

Por último, podemos correr varios experimentos con distintos hiperparámetros (en este caso el único hiperparámetro que importa es el número de capas ocultas) para ver si la hipótesis se puede rechazar o no. Para eso podemos utilizar distintos métodos, en lo personal prefiero hacerlo directamente via BASH.

In [ ]:
%%bash

LAYERS="[] [64] [64,64] [64,64,64]"

for layers in $LAYERS
do
    python -m mlflow_hydra.experiment \
        input.data_file=./data/wines-data.csv \
        input.run_name=\"layers:$layers\" \
        train.model.layers=$layers
done

<h1 style="text-align:center;">¡Muchas Gracias!</h1>

<h2 style="text-align:center;">¿Preguntas?</h2>

* Twitter: https://twitter.com/crscardellino
* LinkedIn: https://www.linkedin.com/in/crscardellino
* Página Personal: https://crscardellino.ar
* GitHub: https://github.com/crscardellino/
* Código de la presentación: https://github.com/crscardellino/data-ar-mlflow-hydra